<a href="https://colab.research.google.com/github/halldm2000/NOAA-AI-2020-TUTORIAL/blob/master/03_FullyConnected/curve_fitting_neural_net_2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Curve Fitting with PyTorch, 2D


**Download Data**

In [ ]:
source = "ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/daily/data/N_seaice_extent_daily_v3.0.csv"
dest    = "/content/data/"
!wget  -c -e robots=off -P {dest} {source}

**Data Loading Routine**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import torch, numpy as np

# make directory to store images
!mkdir -p images

def load_data():
  global xscaler, yscaler, X,Y

  # read sea-ice extent data from CSV file
  csv    = "/content/data/N_seaice_extent_daily_v3.0.csv"
  data   = np.loadtxt(csv, skiprows=2, delimiter=',', usecols=(0,1,2,3,4))
  extent = data[:,3].astype("float32")
  year   = data[:,0].astype("float32")
  nrows  = data.shape[0]

  # get day of the year
  yday   = np.zeros_like(year) 
  for i in range(nrows):
    yr, month, day = data[i,0:3].astype(int)
    yday[i] = datetime(yr,month,day).timetuple().tm_yday
  
  # put variables in column format
  yday  = np.reshape(yday,  (-1,1))
  year  = np.reshape(year,  (-1,1))
  extent= np.reshape(extent,(-1,1))

  Y = extent
  X = np.hstack( (year,yday))

  # fir data in range [-1,1]
  xscaler = StandardScaler()
  yscaler = StandardScaler()
  x = torch.tensor(xscaler.fit_transform(X))
  y = torch.tensor(yscaler.fit_transform(Y))
  return x,y
  
def shuffle_blocks(blocksize = 365):
  global x,y,X,Y
  
  nrows = len(x)//blocksize
  inds  = torch.arange(0,nrows*blocksize)
  inds  = inds.view(nrows, blocksize)
  inds  = inds[torch.randperm(nrows),:].flatten()
  x,y,X,Y = x[inds], y[inds], X[inds], Y[inds]
  

**Plotting Routine**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot(interval = 1):

  global X,Y, X_train, Y_train, X_val, Y_val

  # exit if it's not time to plot
  if epoch%interval != 0: return

  if(epoch==0):
    X_train = xscaler.inverse_transform(x_train)
    Y_train = yscaler.inverse_transform(y_train)
    X_val   = xscaler.inverse_transform(x_val)
    Y_val   = yscaler.inverse_transform(y_val)

  # plot observations
  fig = plt.figure(figsize=(15,10),dpi=72*1.25)

  ax = fig.add_subplot(111, projection='3d')#, proj_type = 'ortho')
  ax.scatter(X[:,0], X[:,1], Y, c=X[:,0], marker='.',alpha=0.5)
  ax.set_xlabel('year')
  ax.set_ylabel('day')
  ax.set_zlabel('million sq km', rotation=90)
  ax.view_init(30, 30)

  prediction = model(x)
  Y_pred = yscaler.inverse_transform(prediction)
  ax.scatter(X[:,0], X[:,1], Y_pred,c='black',s=1, alpha=0.5)
  ax.set_zlim(Y.min(), Y.max())

  # set title, axis labels
  plt.title(f"Annual Artic Sea-ice Extent   Epoch={epoch}   Validation loss={val_loss[-1]:.4f}",fontsize=14)
  ax.set_xlabel('year')
  ax.set_ylabel('day of the year')
  ax.set_zlabel('million sq km')

  # plot train and validation loss
  #plt.subplot(1,2,2)
  #epochs = range( len(train_loss) )
  #plt.semilogy(epochs, train_loss, label="training loss")
  #plt.semilogy(epochs, val_loss,'g', label="validation loss")
  #plt.xlabel('epoch')
  #plt.ylabel('average error')
  #plt.ylim(bottom=5e-3)
  #plt.legend()

  # save figure to file
  plt.savefig(f'./images/img_{epoch//interval:04d}',bbox_inches='tight');
  plt.show()

**Training and Validation**

In [ ]:
import torch, torch.nn as nn, numpy as np
torch.manual_seed(0)

# DATA
x,y = load_data()
shuffle_blocks(blocksize=100)
ntrain, nval, ntest = (len(x)*torch.tensor([0.50, 0.25, 0.25])).int()
x_train, x_val, xtest = x[:ntrain], x[ntrain:-ntest], x[-ntest:]
y_train, y_val, ytest = y[:ntrain], y[ntrain:-ntest], y[-ntest:]

# MODEL
N = 30
model = nn.Sequential(nn.Linear(2,N), nn.ReLU(), nn.Linear(N,1))

# CONFIGURE
optimizer = torch.optim.AdamW(params = model.parameters(), lr=1e-2)
loss_fcn  = torch.nn.MSELoss()

train_loss, val_loss=[],[]
for epoch in range(200+1):

  # TRAIN
  prediction = model(x_train)
  optimizer.zero_grad()
  loss = loss_fcn(prediction, y_train)
  loss.backward()
  optimizer.step()
  train_loss.append(loss.item())

  # VALIDATE
  with torch.no_grad():

    p_val = model(x_val)
    loss_val = loss_fcn(p_val, y_val)
    val_loss.append(loss_val.item())
    plot(interval=1)


**Combine images into a video**

In [ ]:
!ffmpeg -loglevel warning -i ./images/img_%04d.png -vf scale=1280:-2 -pix_fmt yuv420p -y out.mp4

**Display video in the notebook**

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=1200 controls><source src="%s" type="video/mp4"></video>
""" % data_url)

In [ ]:
# remove images
!rm ./images/img_*.png